# Exploration de dataset

In [3]:
import pandas as pd
df=pd.read_csv("transactions_1.csv")

In [11]:
df.head(5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,845717,2019-12-14 19:51:57,213156747557083,fraud_Gutmann McLaughlin and Wiza,home,79.45,Adam,Santos,M,725 Jo Trace Apt. 102,...,34.1556,-118.2322,172817,Advertising account planner,1982-07-30,7c84c214fc4083a31e60b410062db6a6,1355514717,34.410611,-118.001926,1
1,56861,2020-07-11 05:27:50,4481131401752,fraud_Pfeffer LLC,grocery_net,59.10,Frank,Foster,M,37910 Ward Lights,...,42.2848,-71.7205,35299,English as a second language teacher,1975-04-30,46decc0503eab463f10a7723b0aeb7df,1373520470,43.098728,-71.694259,1
2,516292,2020-12-22 19:00:05,213186334552049,fraud_Wintheiser Dietrich and Schimmel,misc_pos,2.33,Andrea,Perkins,F,37426 Olson Villages,...,41.8117,-79.2643,528,Chief Marketing Officer,1972-09-13,5912f519966bfe2fcc3c475863e3d167,1387738805,42.382939,-79.012416,0
3,524872,2019-08-14 07:38:52,4067137330196900,fraud_Bernhard Inc,gas_transport,34.19,Christina,Eaton,F,3256 Brooks Field,...,33.9315,-87.6194,1186,Politician's assistant,1986-11-12,8de346ae3de5c6d851050ad47cce2dee,1344929932,33.599369,-87.240597,1
4,1047572,2020-03-10 08:51:47,6517217825320610,fraud_Connelly Reichert and Fritsch,gas_transport,54.32,James,Reese,M,26975 Richardson Mills Apt. 402,...,31.6453,-90.1801,1196,Librarian_academic,1958-06-11,05275fc06273d00eb721adb412912b06,1362905507,32.147869,-90.232111,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   trans_date_trans_time  object 
 2   cc_num                 int64  
 3   merchant               object 
 4   category               object 
 5   amt                    float64
 6   first                  object 
 7   last                   object 
 8   gender                 object 
 9   street                 object 
 10  city                   object 
 11  state                  object 
 12  zip                    int64  
 13  lat                    float64
 14  long                   float64
 15  city_pop               int64  
 16  job                    object 
 17  dob                    object 
 18  trans_num              object 
 19  unix_time              int64  
 20  merch_lat              float64
 21  merch_long             float64
 22  is_fraud          

In [13]:
df["is_fraud"].value_counts()

is_fraud
0    1072536
1     779858
Name: count, dtype: int64

# Division de données en chunks

In [16]:
import os
import pandas as pd

def split_csv_by_parts(input_file, output_dir, num_parts):
    """
    Divise un fichier CSV en un nombre spécifié de fichiers.

    :param input_file: Chemin complet vers le fichier CSV source.
    :param output_dir: Répertoire où sauvegarder les morceaux.
    :param num_parts: Nombre de morceaux souhaités.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Compter le nombre total de lignes
    total_rows = sum(1 for _ in open(input_file)) - 1  # Exclure l'en-tête

    # Calculer la taille de chaque chunk
    chunk_size = total_rows // num_parts

    # Lire le fichier par chunks et écrire les morceaux
    for i, chunk in enumerate(pd.read_csv(input_file, chunksize=chunk_size)):
        output_file = os.path.join(output_dir, f"part_{i + 1}.csv")
        chunk.to_csv(output_file, index=False)
        print(f"Chunk {i + 1} sauvegardé : {output_file}")

# Exemple d'utilisation
input_csv = "transactions_1.csv"
output_directory = "csv_batches"
num_splits = 10  # Nombre de fichiers souhaités

split_csv_by_parts(input_csv, output_directory, num_splits)

Chunk 1 sauvegardé : csv_batches/part_1.csv
Chunk 2 sauvegardé : csv_batches/part_2.csv
Chunk 3 sauvegardé : csv_batches/part_3.csv
Chunk 4 sauvegardé : csv_batches/part_4.csv
Chunk 5 sauvegardé : csv_batches/part_5.csv
Chunk 6 sauvegardé : csv_batches/part_6.csv
Chunk 7 sauvegardé : csv_batches/part_7.csv
Chunk 8 sauvegardé : csv_batches/part_8.csv
Chunk 9 sauvegardé : csv_batches/part_9.csv
Chunk 10 sauvegardé : csv_batches/part_10.csv
Chunk 11 sauvegardé : csv_batches/part_11.csv


# Envoie De chunks vers répertoire "/home/master/kafka_ingestion/"

In [17]:
import os
import shutil

# Chemin des fichiers CSV à déplacer
source_directory = "/home/master/csv_batches/"
destination_directory = "/home/master/kafka_ingestion/"

# Liste des fichiers dans le répertoire source
files = sorted([f for f in os.listdir(source_directory) if f.endswith(".csv")])

# Vérifier si le répertoire de destination existe
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

print("Appuyez sur Entrée pour envoyer chaque fichier un par un.")

for file in files:
    # Chemin complet source et destination
    source_path = os.path.join(source_directory, file)
    destination_path = os.path.join(destination_directory, file)
    
    # Attendre l'entrée de l'utilisateur
    input(f"Prêt à envoyer {file}? Appuyez sur Entrée pour continuer...")
    
    # Déplacer le fichier
    shutil.move(source_path, destination_path)
    print(f"Fichier {file} envoyé vers {destination_directory}.")
    
print("Tous les fichiers ont été envoyés.")


Appuyez sur Entrée pour envoyer chaque fichier un par un.


Prêt à envoyer part_1.csv? Appuyez sur Entrée pour continuer... 


Fichier part_1.csv envoyé vers /home/master/kafka_ingestion/.


Prêt à envoyer part_10.csv? Appuyez sur Entrée pour continuer... 


Fichier part_10.csv envoyé vers /home/master/kafka_ingestion/.


...

Fichier part_7.csv envoyé vers /home/master/kafka_ingestion/.


Prêt à envoyer part_8.csv? Appuyez sur Entrée pour continuer... 


Fichier part_8.csv envoyé vers /home/master/kafka_ingestion/.


Prêt à envoyer part_9.csv? Appuyez sur Entrée pour continuer... 


Fichier part_9.csv envoyé vers /home/master/kafka_ingestion/.
Tous les fichiers ont été envoyés.
